In [4]:
# 生成一个超大的三维数组，测试 carta 是如何传输文件的
from importlib.util import find_spec
from astropy.io import fits
import numpy as np
fn = r'F:\_todo\dsrt\forimage\data\ODACH_DSRT02_SRIM_L2_150.900000MHz_20210829095000.fits'
hdu = fits.open(fn)
arr = hdu[1].data['STOKESI']
a = arr
for i in range(0,80):

    arr = np.concatenate((a,arr))

arr.shape

#生成主HDU
p_hdu = fits.PrimaryHDU()
header = p_hdu.header

p_hdu = fits.PrimaryHDU(data=arr,header=header)

hdul = fits.HDUList([p_hdu])


hdul.writeto('proj.fits')


In [8]:
# 生成一个超大的二维数组，测试 carta 怎么处理大的二维文件

from importlib.util import find_spec
from astropy.io import fits
import numpy as np
fn = r'F:\_todo\dsrt\forimage\data\ODACH_DSRT05_SRSP_L1_05M_20220212081001_V01.01.fits'
hdu = fits.open(fn)
arr = hdu[0].data

# arr.shape
a = arr
for i in range(0,18):

    arr = np.concatenate((a,arr),2)

arr.shape

#生成主HDU
p_hdu = fits.PrimaryHDU()
header = p_hdu.header

p_hdu = fits.PrimaryHDU(data=arr,header=header)

hdul = fits.HDUList([p_hdu])

hdul.writeto('spebigtest.fits')

In [ ]:
# 查看频谱数据的时间间隔

from astropy.io import fits

fn = r'F:\_todo\dsrt\forimage\data\ODACH_DSRT05_SRSP_L1_05M_20220212081001_V01.01.fits'

hdu = fits.open(fn)

header = hdu[0].header

print ('Time interval : ',header['CDELT1'])

arr = hdu[1].data['time']

for i in range(arr.shape[1]-1):
    print(arr[0][i+1]-arr[0][i],'  ',i)




In [ ]:
#  插值法补充间隔

from astropy.io import fits

fn = r'F:\_todo\dsrt\forimage\data\ODACH_DSRT05_SRSP_L1_05M_20220212081001_V01.01.fits'

hdu = fits.open(fn)

interval = hdu[0].header['CDELT1']

header = hdu[0].header

header



In [ ]:
def spe_merge(fitslist, start_times=None, end_times=None):
    '''
    根据传入的HDU列表与起止时间合并频谱数据,

    Args:
    fitslist: HDU 列表 类型为 astropy.io.fits.hdu.hdulist.HDUList,传入时
    应保证列表中的文件的时间顺序
    start_times : 开始时间，默认为 None 不指定该参数或传入不合法参数设为第一个文件的第一个像素为起点
    start_times : 结束时间，默认为 None 不指定该参数或传入不合法参数设为最后一个文件的最后一个像素为终点
    '''

    # 设置零点
    dtime0 = datetime.datetime.strptime("00:00:00", "%H:%M:%S")
    start_time = None
    end_time = None

    # 时间转换，由字符串转换为毫秒数
    if (start_times != None):
        start_time = datetime.datetime.strptime(start_times, "%H:%M:%S")
        start_time = int(format((start_time - dtime0).seconds))

    if (end_times != None):
        end_time = datetime.datetime.strptime(end_times, "%H:%M:%S")
        end_time = int(format((end_time - dtime0).seconds))

    # 空文件列表返回错误
    if len(fitslist) == 0:
        return 'empyt list'

    # 获取数据的空间维度
    try:
        data_y = fitslist[0][0].header['NAXIS2']
    except:
        return False

    # 未指定具体时间或设置时间超出实际范围则置为默认起止时间
    if start_time == None or start_time < fitslist[0][1].data['TIME'][0][0] or start_time > \
            fitslist[-1][1].data['TIME'][0][-1]:
        start_time = fitslist[0][1].data['TIME'][0][0]
    if end_time == None or end_time > fitslist[-1][1].data['TIME'][0][-1] or end_time < fitslist[0][1].data['TIME'][0][
        0]:
        end_time = fitslist[-1][1].data['TIME'][0][-1]

    '''将待处理文件的数据导入新的数组'''

    # 导入时间矩阵和频点矩阵
    # 频点矩阵
    freArray = fitslist[0][1].data['FREQUENCY']

    # 时间矩阵

    otimeArray = fitslist[0][1].data['TIME']

    for i in range(1, len(fitslist)):
        otimeArray = np.hstack((otimeArray, fitslist[i][1].data['TIME']))

    # 数据矩阵

    odataArray = fitslist[0][0].data

    for i in range(1, len(fitslist)):
        odataArray = np.concatenate((odataArray, fitslist[i][0].data), axis=2)

    # 切割

    # 确定时间范围
    st_index = np.where(otimeArray[0] >= start_time)[0][0]
    en_index = np.where(otimeArray[0] >= end_time)[0][0]

    ndata = odataArray[:, :, st_index:en_index + 1]
    timeArray = otimeArray[:, st_index:en_index + 1]

    # 生成主表
    nPrimaryHdu = fits.PrimaryHDU()
    header = fitslist[0][0].header
    

    # 生成扩展表 format = D （双精浮点数）
    timeTable = fits.Column(name='TIME', array=timeArray, format='%dD' % (timeArray.size))
    freTable = fits.Column(name='FREQUENCY', array=freArray, format='%dD' % (freArray.size))

    nBinTable = fits.BinTableHDU.from_columns([timeTable, freTable])

    # 新的HUDlist
    nHdu = fits.HDUList([nPrimaryHdu, nBinTable])

    return nHdu